In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split

import numpy as np
import pandas as pd

from utils.path_utils import Files
from neural.config import *
from evaluation.analysis import Plotter

from neural.dataset import numpyDataset
from neural.mlp import MLP, MlpBlock, RMLP
from neural.training import Trainer, MLPTrainer
from neural.train_results import FitResult

pd.options.display.max_columns = 30

torch.set_printoptions(sci_mode=False)

In [ ]:
class_name = "MLP"
params = {'in_dim': 2,
 'dims': [3],
 'nonlins': ['none']
 }
MLP.__init__.__annotations__
globals()[class_name](**params)

In [ ]:
MlpBlock(2, [3,4,5], ['relu']*3)

In [ ]:
seed = 42
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [ ]:
dataset_config = DatasetConfig(
    input_frames = [0,-3,-15,-18,-30], # list
    pred_frames = [12], # list
    log_path = './data/Exp1_GuyGilad_logs_yolo/bboxes2.csv', # str
)

In [ ]:
train_config = TrainConfig(
    seed = 42, # int
    dataset = None, # Dataset
    model = None, # Module
    loss_fn = None, # Union
    optimizer = None, # Union
    device = f"cuda", # str
    num_epochs = 100, # int
    checkpoints = None, # str
    early_stopping = (None,), # int
    print_every = (5,), # int
    learning_rate = 0.001, # float
    weight_decay = 1e-05, # float
    batch_size = 64, # int
    shuffle = True, # bool
    num_workers = 0, # int
    train_test_split = 0.8, # float
)

In [ ]:
dataset = numpyDataset.create_from_config(dataset_config, save_path=None)

In [ ]:
dataset[:10]

In [ ]:
# Split the dataset
ds_train, ds_test = random_split(dataset, [0.8, 0.2])

In [ ]:
# Create the dataloaders
dl_train = DataLoader(ds_train, batch_size=256, shuffle=True)
dl_test = DataLoader(ds_test, batch_size=256, shuffle=True)

In [ ]:
# Current Best Model
hidden_dims = [120, 40, 12, 4, 2]
activaations = ['relu'] * (len(hidden_dims))# + ['softmax']
in_dim = ds_train[0][0].shape[0]
model = MLP(in_dim=in_dim, dims=hidden_dims, nonlins=activaations)


In [ ]:
block_in_dim = 40
# block_dims = [10, 4, 10, 40]
block_dims = [10, 40]
block_activations = ['relu'] * (len(block_dims))
in_dim = 4 * len(dataset_config.input_frames)
out_dim = 2*len(dataset_config.pred_frames)

model = RMLP(in_dim=in_dim, block_in_dim=block_in_dim, block_dims=block_dims, block_nonlins=block_activations, n_blocks=4, out_dim=out_dim)

In [ ]:
print(model)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

In [ ]:
trainer = MLPTrainer(model, loss_fn, optimizer, device=device)

In [ ]:
trainer.fit(dl_train=dl_train, dl_test=dl_test, num_epochs=1000, checkpoints='logs/RecurrentMLP4.pt', print_every=1, log=True)

In [ ]:
model.save

# Dataset Creation

In [ ]:
timing_config = TimingConfig.load_json("/Users/guycohen/Desktop/Bio-Project/time_config.json")

log_path = "/Users/guycohen/Desktop/Bio-Project/eval/bboxes2.csv"


pltr = Plotter(log_path, timing_config)
data = pltr._data.copy()

In [ ]:
df = pd.DataFrame(index=range(8), columns=['x', 'y'])
df.iloc[0]  = [1, 0]
df

In [ ]:
data.iloc[[0,-1]]


In [ ]:
# data.ag
data.rolling(window=10, method='table').flatten()

In [ ]:
np.arange(40).reshape(8,5)[[2,3]].flatten()

In [ ]:
data2 = data.dropna(inplace=False, ignore_index=False, axis=0)
mask = data.isna().any(axis=1)
data['']

In [ ]:
# make sure that 0 is first at frames_for_pred
def create_dataset(log_path: str, frames_for_pred=[0,-3,-15], pred_frame=7, save_path:str=None) -> Dataset:
    data = pd.read_csv(log_path)
    start_idx = abs(min(frames_for_pred)) + 1
    X_mask = np.asanyarray(frames_for_pred)
    y_mask = np.asanyarray([pred_frame])

    data = Plotter.concat(data, Plotter.calc_centers(data, "wrm"))
    
    wrm_centers = data[['wrm_center_x', 'wrm_center_y']].to_numpy(dtype=np.float64)
    wrm_boxes = data[['wrm_x', 'wrm_y', 'wrm_w', 'wrm_h']].to_numpy(dtype=np.float64)
    
    # Create columns for X and y
    cols = ['wrm_x', 'wrm_y', 'wrm_w', 'wrm_h']
    y_cols = ['wrm_center_x', 'wrm_center_y']
    X_cols = []
    for i in frames_for_pred:
        X_cols += [col+str(i) for col in cols]
    
    # Create X and y
    X = pd.DataFrame(index=data.index, columns=X_cols)
    y = pd.DataFrame(index=data.index, columns=y_cols)
    for i in range(start_idx, len(data)-pred_frame-1):
        X.iloc[i] = wrm_boxes[i + X_mask].reshape(1, -1)
        y.iloc[i] = wrm_centers[i + y_mask].reshape(1, -1)

    # Drop rows with NaN values
    na_mask = np.ma.mask_or(X.isna().any(axis=1), y.isna().any(axis=1))
    X = X.loc[~na_mask]
    y = y.loc[~na_mask]
    
    X = X.to_numpy(dtype=np.float32, copy=True)
    y = y.to_numpy(dtype=np.float32, copy=True)

    # make X and y coordinates relative to the prediction frame
    x_cord_mask = np.arange(X.shape[1]) % 4 == 0
    y_cord_mask = np.arange(X.shape[1]) % 4 == 1
    
    x_cords = X[:, 0].reshape(-1, 1)
    y_cords = X[:, 1].reshape(-1, 1)

    y[:, [0]] -= x_cords
    y[:, [1]] -= y_cords
    X[:, x_cord_mask] -= x_cords#
    X[:, y_cord_mask] -= y_cords#.reshape(-1, 1)
    
    if save_path is not None:
        np.save(save_path + "dataset_X_3.npy", X, allow_pickle=True)
        np.save(save_path + "dataset_y_3.npy", y, allow_pickle=True)
        # X.to_csv(save_path + "dataset_X.csv", index=False)
        # y.to_csv(save_path + "dataset_y.csv", index=False)

    return numpyDataset(X, y)

In [ ]:
# log_path = "/Users/guycohen/Desktop/Bio-Project/eval/bboxes2.csv"
log_path = f"D:/Guy_Gilad/Exp1_GuyGilad/logs_yolo/bboxes.csv"
pred_frame=7
frames_for_pred=[0,-3,-15,-18,-30]
dataset = create_dataset(log_path, save_path="D:\Guy_Gilad\Bio-Proj\evaluation", frames_for_pred=frames_for_pred, pred_frame=pred_frame)

In [ ]:
X, y = ds[:]
df = pd.DataFrame({"dx": y[:,0], "dy": y[:,1]})

In [ ]:
df.hist()